In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
import numbers

In [2]:
times = pd.read_csv(r"C:\Users\Gio\OneDrive\Desktop\Bicocca\FoCS\datsets\timesData.csv")

In [3]:
times['num_students'] = times['num_students'].str.split(",")
times['num_students'] = times['num_students'].str.join("")

In [4]:
times['income'] = times['income'].replace('-', np.nan)

1) For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [5]:
timesmin = times.loc[times.groupby('university_name', as_index=False)['year'].idxmin()]
timesmin.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569,17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,17422,15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,16099,24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895,13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,9252,19.2,18%,48 : 52,2012


In [6]:
timesmax = times.loc[times.groupby('university_name', as_index=False)['year'].idxmax()]
timesmax.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569,17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252,19.2,18%,48 : 52,2016


2) For each university, compute the improvement in income between the least recent and the most recent data points

In [7]:
times_delta = pd.merge(timesmax, timesmin, on = 'university_name')

In [8]:
times_delta.dropna(subset=['income_x','income_y'], inplace = True)

In [9]:
times_delta['incomemax'] = pd.to_numeric(times_delta['income_x'])
times_delta['incomemin'] = pd.to_numeric(times_delta['income_y'])
times_delta['differenza'] = times_delta['incomemax'] - times_delta['incomemin']
risultato_2 = times_delta[['university_name','incomemax','incomemin','differenza']]
risultato_2.head()

,university_name,incomemax,incomemin,differenza
1,Aalborg University,43.7,36.4,7.3
2,Aalto University,61.6,61.9,-0.3
3,Aarhus University,68.3,61.5,6.8
4,Aberystwyth University,31.3,35.5,-4.2
5,Adam Mickiewicz University,28.7,28.7,0.0


3) Find the university with the largest increase computed in the previous point

In [10]:
risultato_2.loc[risultato_2['differenza'].idxmax()]

university_name    TU Dresden
incomemax                99.7
incomemin                31.9
differenza               67.8
Name: 428, dtype: object

4) For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range)

In [11]:
cwur = pd.read_csv(r"C:\Users\Gio\OneDrive\Desktop\Bicocca\FoCS\datsets\cwurData.csv")

In [12]:
shg = pd.read_csv(r"C:\Users\Gio\OneDrive\Desktop\Bicocca\FoCS\datsets\shanghaiData.csv")

In [13]:
cwur.rename(columns={'institution':'university_name'},inplace = True)

In [14]:
shgmax = shg.iloc[shg.groupby('university_name')['year'].idxmax()]
cwurmax = cwur.iloc[cwur.groupby('university_name')['year'].idxmax()]

In [15]:
unimerge = shgmax.merge(timesmax.merge(cwurmax, on = 'university_name'), on = 'university_name')
wranks = unimerge[['university_name','world_rank','world_rank_x','world_rank_y']]

In [16]:
splitted_1 = wranks["world_rank"].str.split(r"-|=", expand=True)              # Pulizia dei caratteri = e - per la colonna world_rank ovvero colonna relativa a times
splitted_1[1].fillna(splitted_1[0],inplace= True)
splitted_1[2] = pd.to_numeric(splitted_1[0])
splitted_1[3] = pd.to_numeric(splitted_1[1])
splitted_1[4] = (splitted_1[2] + splitted_1[3])/2
wranks['avg wr'] = splitted_1[4].astype('float32')

In [17]:
splitted_2 = wranks["world_rank_x"].str.split(r"-|=", expand=True)        # Pulizia dei caratteri = e - per la colonna world_rank_x ovvero colonna relativa a shanghai
splitted_2[0].replace("",splitted_2[1], inplace = True)
splitted_2[1].fillna(splitted_2[0],inplace= True)
splitted_2[2] = pd.to_numeric(splitted_2[0])
splitted_2[3] = pd.to_numeric(splitted_2[1])
splitted_2[4] = (splitted_2[2] + splitted_2[3])/2
wranks['avg wr_x'] = splitted_2[4].astype('float32')

In [18]:
wranks.rename(columns={'world_rank_y': 'wr_cwur', 'avg wr': 'wr_times','avg wr_x':'wr_shg'}, inplace=True)

In [19]:
wranks['diff_c-t'] = np.absolute(wranks['wr_cwur'] - wranks['wr_times'])     # differenze, in valore assoluto, tra i diversi rankings 
wranks['diff_c-s'] = np.absolute(wranks['wr_cwur'] - wranks['wr_shg'])
wranks['diff_t-s'] = np.absolute(wranks['wr_times'] - wranks['wr_shg'])
wranks = wranks[['university_name','diff_c-t','diff_c-s','diff_t-s']]
wranks['HighScore'] = wranks[['diff_c-t','diff_c-s','diff_t-s']].max(axis=1) # aggiunta della colona con i valori massimi della differenza
wranks.head()

,university_name,diff_c-t,diff_c-s,diff_t-s,HighScore
0,Aalborg University,214.5,339.5,125.0,339.5
1,Aalto University,29.5,145.5,175.0,175.0
2,Aarhus University,49.0,16.0,33.0,49.0
3,Aix-Marseille University,80.0,69.5,149.5,149.5
4,Aristotle University of Thessaloniki,8.5,241.5,250.0,250.0


5) Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [20]:
times[['female_male_ratio','num_students']] = times[['female_male_ratio','num_students']].replace('-', np.nan)

In [21]:
times_ratio = times.dropna(subset=['female_male_ratio','num_students'])

In [22]:
tr_max = times_ratio.loc[times_ratio.groupby('university_name')['year'].idxmax()]
tr_max = tr_max[['university_name','country','num_students','female_male_ratio']]

In [23]:
split = tr_max["female_male_ratio"].str.split(r":", expand=True)  # eliminazione carattere :
split[2] = pd.to_numeric(split[0])
split[3] = pd.to_numeric(split[1])
tr_max['f_ratio'] = split[2].astype('int32')
tr_max['m_ratio'] = split[3].astype('int32')
tr_max['num_students'] = tr_max['num_students'].astype('int64')

In [24]:
tr_max['num_f'] = tr_max['num_students']*tr_max['f_ratio']/100
tr_max['num_m'] = tr_max['num_students']*tr_max['m_ratio']/100
tr_max[["num_f", "num_m"]] = np.round(tr_max[["num_f", "num_m"]], 0)
tr_max.head()

,university_name,country,num_students,female_male_ratio,f_ratio,m_ratio,num_f,num_m
2003,Aalborg University,Denmark,17422,48 : 52,48,52,8363.0,9059.0
2056,Aalto University,Finland,16099,32 : 68,32,68,5152.0,10947.0
1908,Aarhus University,Denmark,23895,54 : 46,54,46,12903.0,10992.0
2105,Aberystwyth University,United Kingdom,9252,48 : 52,48,52,4441.0,4811.0
2404,Adam Mickiewicz University,Poland,40633,71 : 29,71,29,28849.0,11784.0


6) Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [25]:
tr_max['f/m_ratio'] = (tr_max['f_ratio']/tr_max['m_ratio'])

In [26]:
tr_max[tr_max['f/m_ratio'].astype(str).str.contains('inf')] #pulizia del valore inf

,university_name,country,num_students,female_male_ratio,f_ratio,m_ratio,num_f,num_m,f/m_ratio
2227,Ewha Womans University,South Korea,17625,100 : 0,100,0,17625.0,0.0,inf


In [27]:
tr_max_6 = tr_max.drop([2227])

In [28]:
media = tr_max_6[['f/m_ratio']].mean()
media

f/m_ratio    1.081102
dtype: float64

In [29]:
below_avg = tr_max_6[tr_max_6['f/m_ratio'] < 1.081102]
below_avg

,university_name,country,num_students,female_male_ratio,f_ratio,m_ratio,num_f,num_m,f/m_ratio
2003,Aalborg University,Denmark,17422,48 : 52,48,52,8363.0,9059.0,0.923077
2056,Aalto University,Finland,16099,32 : 68,32,68,5152.0,10947.0,0.470588
2105,Aberystwyth University,United Kingdom,9252,48 : 52,48,52,4441.0,4811.0,0.923077
2406,Ajou University,South Korea,12706,33 : 67,33,67,4193.0,8513.0,0.492537
2408,Alexandria University,Egypt,127431,46 : 54,46,54,58618.0,68813.0,0.851852
...,...,...,...,...,...,...,...,...,...
2104,Zhejiang University,China,47508,41 : 59,41,59,19478.0,28030.0,0.694915
1856,École Normale Supérieure,France,2400,46 : 54,46,54,1104.0,1296.0,0.851852
2013,École Normale Supérieure de Lyon,France,2218,49 : 51,49,51,1087.0,1131.0,0.960784
1904,École Polytechnique,France,2429,18 : 82,18,82,437.0,1992.0,0.219512


7) For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [30]:
num = below_avg.groupby('country').sum()[['num_students']]
num

,num_students
country,
Australia,146235
Austria,61033
Bangladesh,62716
Brazil,278774
Canada,86779
Chile,90951
China,885293
Colombia,54645
Czech Republic,38685


In [31]:
timesmax['n_stud'] = pd.to_numeric(timesmax['num_students'])

In [32]:
den = timesmax.groupby('country').sum()[['n_stud']]
den

,n_stud
country,
Argentina,108373.0
Australia,743627.0
Austria,134477.0
Bangladesh,62716.0
Belarus,29303.0
...,...
Unisted States of America,0.0
United Arab Emirates,14447.0
United Kingdom,1324842.0


In [33]:
risultato_7 = pd.merge(num,den,on='country')

In [34]:
risultato_7['totale_stud'] = risultato_7['n_stud'].astype('int64')
ris_7 = risultato_7.drop(risultato_7[['n_stud']], axis=1)

In [35]:
ris_7['ratio'] = (ris_7['num_students']/ris_7['totale_stud'])
ris_7

,num_students,totale_stud,ratio
country,,,
Australia,146235,743627,0.196651
Austria,61033,134477,0.453855
Bangladesh,62716,62716,1.000000
Brazil,278774,534688,0.521377
Canada,86779,717054,0.121022
Chile,90951,116026,0.783885
China,885293,1285619,0.688612
Colombia,54645,54645,1.000000
Czech Republic,38685,203321,0.190266


8) Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [36]:
edu = pd.read_csv(r"C:\Users\Gio\OneDrive\Desktop\Bicocca\FoCS\datsets\educational_attainment_supplementary_data.csv")
edu.tail()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
edu[['country_name','series_name', '1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015']] = edu[['country_name','series_name', '1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015']].replace('NaN', np.nan)
edu.dropna(subset=['country_name','series_name'], inplace=True)
edu.tail()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN
79049,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.14588,NaN,NaN


9) From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [38]:
edu_9 = pd.melt(edu, id_vars = ['country_name','series_name'], value_vars = ['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'])
edu_9.head()

,country_name,series_name,variable,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


10) For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [39]:
T = list (timesmax['university_name'])
for elem in list (timesmax['university_name']):
    print (elem)

AGH University of Science and Technology
Aalborg University
Aalto University
Aarhus University
Aberystwyth University
Adam Mickiewicz University
Aix-Marseille University
Ajou University
Alexandria University
Alexandru Ioan Cuza University
Aligarh Muslim University
American University
American University of Beirut
American University of Sharjah
Amirkabir University of Technology
Amrita University
Anadolu University
Andhra University
Aristotle University of Thessaloniki
Arizona State University
Asia University, Taiwan
Aston University
Athens University of Economics and Business
Auburn University
Auckland University of Technology
Austral University of Chile
Australian National University
Autonomous University of Barcelona
Autonomous University of Madrid
Babeş-Bolyai University
Bangor University
Bar-Ilan University
Bauman Moscow State Technical University
Bayreuth University
Beijing Institute of Technology
Belarusian State University
Ben-Gurion University of the Negev
Bielefeld University


In [40]:
S = list (shgmax['university_name'])
for elem in list (shgmax['university_name']):
    print (elem)

Aalborg University
Aalto University
Aarhus University
Aix Marseille University
Aix-Marseille University
Aristotle University of Thessaloniki
Arizona State University
Arizona State University - Tempe
Auburn University
Autonomous University of Barcelona
Autonomous University of Madrid
Bangor University
Bar-Ilan University
Baylor College of Medicine
Beihang University
Beijing Normal University
Beijing University of Aeronautics and Astronautics
Ben-Gurion University of the Negev
Bielefeld University
Birkbeck, University of London
Boston College
Boston University
Brandeis University
Brigham Young University
Brown University
Brunel University
Cairo University
California Institute of Technology
Capital Medical University
Capital University of Medical Sciences
Cardiff University
Carleton University
Carnegie Mellon University
Case Western Reserve University
Catholic University of Chile
Catholic University of Korea
Catholic University of Leuven
Catholic University of Louvain
Catholic University 

In [41]:
C = list (cwurmax['university_name'])
for elem in list (cwurmax['university_name']):
    print (elem)

AGH University of Science and Technology
Aalborg University
Aalto University
Aarhus University
Aberystwyth University
Adam Mickiewicz University in Poznań
Ain Shams University
Aix-Marseille University
Ajou University
Albany Medical College
Albert Ludwig University of Freiburg
Alexandria University
All India Institute of Medical Sciences, New Delhi
American University of Beirut
Amirkabir University of Technology
Ankara University
Aristotle University of Thessaloniki
Arizona State University
Aston University
Auburn University
Australian National University
Autonomous University of Barcelona
Autonomous University of Madrid
Babeș-Bolyai University
Banaras Hindu University
Bangor University
Bar-Ilan University
Baylor College of Medicine
Baylor University
Beihang University
Beijing Institute of Technology
Beijing Jiaotong University
Beijing Normal University
Beijing University of Chemical Technology
Beijing University of Technology
Ben-Gurion University of the Negev
Bielefeld University
Bilk

In [42]:
ris_10 = C + S + T

In [43]:
[[x,ris_10.count(x)] for x in set(ris_10)]

[['Ben-Gurion University of the Negev', 3],
 ['Otto von Guericke University of Magdeburg', 1],
 ['Southwest University', 1],
 ['Indian Institute of Technology Kharagpur', 3],
 ['Tarbiat Modares University', 1],
 ['University of Minho', 2],
 ['TU Dresden', 2],
 ['Second Military Medical University', 1],
 ['Macquarie University', 3],
 ['Eotvos Lorand University', 1],
 ['Huazhong University of Science and Technology', 3],
 ['University of Salento', 2],
 ['Southern Illinois University Carbondale', 1],
 ['University of Texas at Arlington', 2],
 ['University of Bremen', 3],
 ['Gyeongsang National University', 1],
 ['Chalmers University of Technology', 3],
 ['Okayama University', 3],
 ['Tokushima University', 1],
 ['Beijing Institute of Technology', 2],
 ['University of Wisconsin - Milwaukee', 1],
 ['Braunschweig University of Technology', 1],
 ['University of Silesia', 1],
 ['University of Texas Southwestern Medical Center', 1],
 ['The Johns Hopkins University', 1],
 ['University of Technolo

11) In the times ranking, compute the number of times each university appears

In [44]:
ris_11 = times.groupby('university_name').size()
ris_11

university_name
AGH University of Science and Technology    1
Aalborg University                          5
Aalto University                            5
Aarhus University                           6
Aberystwyth University                      5
                                           ..
École Normale Supérieure                    6
École Normale Supérieure de Lyon            6
École Polytechnique                         6
École Polytechnique Fédérale de Lausanne    6
Örebro University                           1
Length: 818, dtype: int64

12) Find the universities that appear at most twice in the times ranking.

In [45]:
ris_11[ris_11 <= 2]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Length: 408, dtype: int64

13) The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [46]:
times_13 = times[(times['year']>=2012) & (times['year']<= 2015)]
shg_13 = shg[(shg['year']>=2012) & (shg['year']<= 2015)]
cwur_13 = cwur[(cwur['year']>=2012) & (cwur['year']<= 2015)]

In [47]:
time_13 = times_13[['world_rank','university_name','year']]
s_13 = shg_13[['world_rank','university_name','year']]
cwr_13 = cwur_13[['world_rank','university_name','year']]

In [48]:
split_time = time_13["world_rank"].str.split(r"-|=", expand=True)        #pulizia times_dataset
split_time[0].replace("",split_time[1], inplace = True)
split_time[1].fillna(split_time[0],inplace= True)
split_time[2] = pd.to_numeric(split_time[0])
split_time[3] = pd.to_numeric(split_time[1])
time_13['rank_min'] = split_time[2]
time_13['rank_max'] = split_time[3]
time_13.rename(columns={'rank_min': 'R_min_time', 'rank_max': 'R_max_time'}, inplace=True)

In [49]:
split_shg = s_13["world_rank"].str.split(r"-|=", expand=True)            #pulizia shanghai_dataset
split_shg[1].fillna(split_shg[0],inplace= True)
split_shg[2] = pd.to_numeric(split_shg[0])
split_shg[3] = pd.to_numeric(split_shg[1])
s_13['rank_min'] = split_shg[2]
s_13['rank_max'] = split_shg[3]
s_13.rename(columns={'rank_min': 'R_min_shg', 'rank_max': 'R_max_shg'}, inplace=True)

In [50]:
shg_2012 = s_13[s_13['year']==2012]
s_2012 = shg_2012[['university_name','year','R_min_shg','R_max_shg']]
time_2012 = time_13[time_13['year']==2012]
t_2012 = time_2012[['university_name','year','R_min_time','R_max_time']]
c_2012 = cwr_13[cwr_13['year']==2012]
#merge
unito_2012 = c_2012.merge(t_2012.merge(s_2012, on='university_name'), on='university_name')

In [52]:
#risultato anno 2012
ris_2012 = unito_2012[(unito_2012['world_rank'] >= unito_2012['R_min_time']) & (unito_2012['world_rank'] >= unito_2012['R_min_shg'])
& (unito_2012['world_rank'] <=  unito_2012['R_max_time']) & (unito_2012['world_rank'] <=  unito_2012['R_max_shg']) ]
ris_2012

,world_rank,university_name,year,year_x,R_min_time,R_max_time,year_y,R_min_shg,R_max_shg


In [53]:
shg_2013 = s_13[s_13['year']==2013]
s_2013 = shg_2013[['university_name','year','R_min_shg','R_max_shg']]
time_2013 = time_13[time_13['year']==2013]
t_2013 = time_2013[['university_name','year','R_min_time','R_max_time']]
c_2013 = cwr_13[cwr_13['year']==2013]
#merge
unito_2013 = c_2013.merge(t_2013.merge(s_2013, on='university_name'), on='university_name')

In [54]:
#risultato anno 2013
ris_2013 = unito_2013[(unito_2013['world_rank'] >= unito_2013['R_min_time']) & (unito_2013['world_rank'] >= unito_2013['R_min_shg'])
& (unito_2013['world_rank'] <=  unito_2013['R_max_time']) & (unito_2013['world_rank'] <=  unito_2013['R_max_shg']) ]
ris_2013

,world_rank,university_name,year,year_x,R_min_time,R_max_time,year_y,R_min_shg,R_max_shg
1,2,Stanford University,2013,2013,2,2,2013,2,2


In [55]:
shg_2014 = s_13[s_13['year']==2014]
s_2014 = shg_2014[['university_name','year','R_min_shg','R_max_shg']]
time_2014 = time_13[time_13['year']==2014]
t_2014 = time_2014[['university_name','year','R_min_time','R_max_time']]
c_2014 = cwr_13[cwr_13['year']==2014]
#merge
unito_2014 = c_2014.merge(t_2014.merge(s_2014, on='university_name'), on='university_name')

In [56]:
#risultato anno 2014
ris_2014 = unito_2014[(unito_2014['world_rank'] >= unito_2014['R_min_time']) & (unito_2014['world_rank'] >= unito_2014['R_min_shg'])
& (unito_2014['world_rank'] <=  unito_2014['R_max_time']) & (unito_2014['world_rank'] <=  unito_2014['R_max_shg']) ]
ris_2014

,world_rank,university_name,year,year_x,R_min_time,R_max_time,year_y,R_min_shg,R_max_shg
88,146,University of Southampton,2014,2014,146,146,2014,101,150
166,320,University of Vermont,2014,2014,301,350,2014,301,400
186,390,Temple University,2014,2014,351,400,2014,301,400


In [57]:
shg_2015 = s_13[s_13['year']==2015]
s_2015 = shg_2015[['university_name','year','R_min_shg','R_max_shg']]
time_2015 = time_13[time_13['year']==2015]
t_2015 = time_2015[['university_name','year','R_min_time','R_max_time']]
c_2015 = cwr_13[cwr_13['year']==2015]
#merge
unito_2015 = c_2015.merge(t_2015.merge(s_2015, on='university_name'), on='university_name')

In [58]:
#risultato anno 2015
ris_2015 = unito_2015[(unito_2015['world_rank'] >= unito_2015['R_min_time']) & (unito_2015['world_rank'] >= unito_2015['R_min_shg'])
& (unito_2015['world_rank'] <=  unito_2015['R_max_time']) & (unito_2015['world_rank'] <=  unito_2015['R_max_shg']) ]
ris_2015

,world_rank,university_name,year,year_x,R_min_time,R_max_time,year_y,R_min_shg,R_max_shg
125,212,Newcastle University,2015,2015,201,225,2015,201,300
134,229,Laval University,2015,2015,226,250,2015,201,300
140,241,Autonomous University of Barcelona,2015,2015,226,250,2015,201,300
152,265,Medical University of Vienna,2015,2015,251,275,2015,201,300
153,269,University of Waterloo,2015,2015,251,275,2015,201,300
193,383,University of Turku,2015,2015,351,400,2015,301,400
